In [1]:
!pip install python-terrier # install pyTerrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ..

In [2]:
!pip install git+https://github.com/allenai/ir_datasets.git@crisisfacts # install ir_datasets (crisisfacts branch)

  Cloning https://github.com/allenai/ir_datasets.git (to revision crisisfacts) to /tmp/pip-req-build-2v4byf9_
  Running command git clone --filter=blob:none --quiet https://github.com/allenai/ir_datasets.git /tmp/pip-req-build-2v4byf9_
  Running command git checkout -b crisisfacts --track origin/crisisfacts
  Switched to a new branch 'crisisfacts'
  Branch 'crisisfacts' set up to track remote branch 'crisisfacts' from 'origin'.
  Resolved https://github.com/allenai/ir_datasets.git to commit e2359e24c9546e2a62284cd1aec6138295bb5ec5
  Preparing metadata (setup.py) ... done
  Created wheel for ir-datasets: filename=ir_datasets-0.5.2-py3-none-any.whl size=304038 sha256=2271825c0704fa1b34e8637eac7dbd7591876d3b15774a89fd6357f0b8bacf85
  Stored in directory: /tmp/pip-ephem-wheel-cache-z6tudes3/wheels/73/e6/f4/2d419bf7da1c4694124abcdaa4a2e58cf103599d3284089045
Successfully built ir-datasets
  Attempting uninstall: ir-datasets
    Found existing installation: ir-datasets 0.5.5
    Uninstalling 

In [3]:
import pyterrier as pt
import json
import os
import requests

In [4]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation

In [5]:
credentials = {
    "institution": "University of Foo", # University, Company or Public Agency Name
    "contactname": "Foo Bar", # Your Name
    "email": "foo@bar.edu", # A contact email address
    "institutiontype": "Research" # Either 'Research', 'Industry', or 'Public Sector'
}

In [6]:
# Write this to a file so it can be read when needed
home_dir = os.path.expanduser('~')

!mkdir -p ~/.ir_datasets/auth/
with open(home_dir + '/.ir_datasets/auth/crisisfacts.json', 'w') as f:
    json.dump(credentials, f)

In [7]:
# Event numbers as a list
eventNoList = [
          "001", # Lilac Wildfire 2017
          "002", # Cranston Wildfire 2018
          "003", # Holy Wildfire 2018
          "004", # Hurricane Florence 2018
          "005", # 2018 Maryland Flood
          "006", # Saddleridge Wildfire 2019
          "007", # Hurricane Laura 2020
          "008" # Hurricane Sally 2020
]

# gpt-2

In [8]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.7 MB/s eta 0:00:00


In [9]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [11]:

def getDaysForEventNo(eventNo):

  # We will download a file containing the day list for an event
  url = "http://trecis.org/CrisisFACTs/CrisisFACTS-"+eventNo+".requests.json"

  # Download the list and parse as JSON
  dayList = requests.get(url).json()

  return dayList

In [17]:
def dataset_pyterier(data_link):
  if not pt.started():
    pt.init()
  data_link = "irds:"+data_link
  print(data_link)
  pyTerrierDataset = pt.get_dataset(data_link)
  indexer = pt.IterDictIndexer("None", type=pt.index.IndexingType(3), meta=['docno', 'text'], meta_lengths=[40, 200])
  index = indexer.index(pyTerrierDataset.get_corpus_iter())
  retriever = pt.BatchRetrieve(index, wmodel="DFReeKLIM", metadata=["docno", "text"])
  return retriever

In [18]:
def generate_summary(query, retriever):
    df=pd.DataFrame(retriever.search(query))
    df = df.sort_values(by=['score'], ascending=False)

    texts = df['text']
    input_text = " ".join(texts)

    input_ids = tokenizer.encode(input_text, max_length=300, return_tensors='pt', truncation=True)

    summary = model.generate(input_ids=input_ids,
                             max_length=350,
                             num_beams=5,
                             no_repeat_ngram_size=2,
                             early_stopping=True
                            )
    summary_text = tokenizer.decode(summary[0], skip_special_tokens=True)
    return(summary_text)

In [19]:
def summary(event,query):
  event_days = getDaysForEventNo(event)
  # Dates for a particular event
  days_list = []
  for d in event_days:
    days_list.append(d["dateString"])
  print(days_list)
  day=str(input())
  event_day_data = 'crisisfacts/'+event+'/'+day

  retriever = dataset_pyterier(event_day_data)
  z = generate_summary(query,retriever)
  print("*********************************************************************************************************************************")
  print('')
  print('Summary:')
  return z

In [21]:
summary("001","injuries")

['2017-12-07', '2017-12-08', '2017-12-09', '2017-12-10', '2017-12-11', '2017-12-12', '2017-12-13', '2017-12-14', '2017-12-15']
2017-12-07
irds:crisisfacts/001/2017-12-07


crisisfacts/001/2017-12-07 documents: 0it [00:00, ?it/s]

12:22:53.302 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


*********************************************************************************************************************************

Summary:


"A San Diego County Sheriff's Department deputy suffered minor injuries while directing traffic. Two civilians are being treated for burn injuries suffered in #LilacFire and are being taken to a hospital, https://t.co/Q2EOJFPVvI Headon collision between Sheriff's vehicle and private vehicle in thick smoke #LilacFire  Moderate damage, no injuries reported UPDATE: #LilacFire now 2000 acres, 0% containment.    Two civilians are being treated for burn injuries suffered in https://t.co/Dp4y94slym Strong winds and low humidity fueling the Lilac Fire burning in Bonsall. To add insult to injury the Red Flag Warni https://t.co/FlpsJPuTSV  CAL FIRE/SAN DIEGO COUNTY FIRE (@CALFIRESANDIEGO) December 8, 2017  Four residents sustained burn injuries as they were evacuating their home, Cal Fire said. Besides the civilian injuries, one firefighter suffered smoke inhalation. It was not clear if it was a factor in the injury. The cause of the fire is under investigation. Firefighters are working to conta

In [16]:
summary("002","buildings collapsing")

['2018-07-25', '2018-07-26', '2018-07-27', '2018-07-28', '2018-07-29', '2018-07-30']
2018-07-25


[INFO] [starting] building docstore
[INFO] [starting] requesting access key
[INFO] [finished] requesting access key s]
docs_iter: 5056doc [00:08, 575.77doc/s]
[INFO] [finished] docs_iter: [00:08] [5056doc] [575.50doc/s]
[INFO] [finished] building docstore [8.79s]


crisisfacts/002/2018-07-25 documents: 0it [00:00, ?it/s]

12:15:44.930 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


*********************************************************************************************************************************

Summary:


'@RivCoSchoolSupt emphasizing the importance of building relationships and supporting SEL in a technology era  https://t.co/oMWVoA2IY6 the guy was just collecting mats to build a base https://t.co/R5wGSofGs7 @sprint yall seriously need to build more cell towers in Rialto  I get no mf service whatsoever. I literally rely on my WiFi Triggr Racing killed it on Henry Millers bike build for Millville  #hinsondomination #hinsonclutch https://t.co/HjGOs8dSzT One of the tankers just flew over my work building after taking off from SBD... leaving a mist of retardant trailing behind it #cranstonfire Iron Spikes coach kevinrussell15 going over some tips on building speed, agility, and explosiveness. In order to be https://t.co/Sd7apxdGcj Last class of the summer... (@ Chaffey College Vocational &amp; Student Support Building in Rancho Cucamonga, CA) https://t.co/8KSZepbmGi In the early hours of the firefight, the flames steadily made headway toward buildings in some sparsely populated areas Wedne